In [1]:
%reload_ext cython

In [227]:
! pip install stop_words

In [2]:
import collections

s1 = "The quick brown fox"
s2 = "Brown fox jumps over the jumps jumps jumps"
s3 = "The the the lazy dog elephant."
s4 = "The the the the the dog peacock lion tiger elephant"

docs = collections.OrderedDict()
docs["s1"] = s1
docs["s2"] = s2
docs["s3"] = s3
docs["s4"] = s4 
for k, v in docs.items():
    print(k,v)
    

s1 The quick brown fox
s2 Brown fox jumps over the jumps jumps jumps
s3 The the the lazy dog elephant.
s4 The the the the the dog peacock lion tiger elephant


In [229]:
len(docs)

4

**Function to make Corpus Matrix**

Function returns a list, 0th element is a 3 dimensional array, each element corresponds to a document in the corpus where the columns are the unique words in that document and the rows are the words in that document (stop words removed).  Second element in list is the column names for each document.

In [3]:
import string
import stop_words
import numpy as np
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer



#Function to make document, word matricies for LDA#
def make_word_matrix(corpus, needToSplit):
    
    
    #Check to make sure NeedToSplit argument is 0 or 1
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    #Define stop words
    stopWords = get_stop_words('english')
    
    
    #Initialize stemmer
    p_stemmer = PorterStemmer()

    
    #Define list to store corpus data#
    c = []
    #Define list to store order of words for each document#
    wordOrder = []
    #Define table to remove punctuation
    table = dict.fromkeys(map(ord, string.punctuation))

    M = len(corpus)
    
    #Check to make sure that dictionary isn't empty#
    if M ==0:
        print("Input dictionary is empty")
        return;
    
    removePunc = string.punctuation
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        if isinstance(i, str) != True:
            print("Corpus input is not a string")
            return;
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
        
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]

        #Remove stop words#
        text = [word for word in words if word not in stopWords]
        # stem tokens
        text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Find number of unique words in each document#
        Vwords = list(set(text))
        wordOrder.append(Vwords)

    #Find unique words in the corpus, this is the vocabulary#    
    wordOrder = list(set(x for l in wordOrder for x in l))
    wordOrder = sorted(wordOrder)

    #Find the number of unique words in the corpus vocabulary#
    V = len(wordOrder)
    
    #For each document in docs, caculate frequency of the words#
    for i in corpus:
        
        #if the documents in the corpus are contained in a single string
        if needToSplit == 1:
            #Remove punctuation 
            text = corpus[i].translate(table)
            #Splits string by blankspace and goes to lower case#
            words = text.lower().split()
            #Remove stop words#
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        else:
            #Remove punctuation
            for j in range(0, len(removePunc)):
                while removePunc[j] in corpus[i]: 
                    corpus[i].remove(removePunc[j])    
            
            #convert everything to a lower case
            corpus[i] = list(map(lambda x:x.lower(),corpus[i]))
            words = corpus[i]
            
            #remove stop words
            text = [word for word in words if word not in stopWords]
            #Stemming
            text = [p_stemmer.stem(i) for i in text]
        #Find total number of words in each document#
        N = len(text)

        #Create matrix to store words for each document#
        wordsMat = np.zeros((N, V))
        count = 0
        for word in text:
            v = wordOrder.index(word)
            wordsMat[count, v] = 1
            count = count + 1
        c.append(wordsMat)

    return [c, wordOrder, M] 

In [4]:
corpusMatrix = make_word_matrix(docs, 1)
corpusMatrix

[[array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
         [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
  array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
         [ 0.,  0.

**Variational Inference**

In [5]:
import numpy as np
import scipy
from scipy import special

**E-Step:** This function uses variational inference to perform the E step in the EM algorithm to estimate the paramteters in the model.  The output of this function are the matricies gamma and phi, where gamma (k vector) is the Dirichlet paramteters and the matrix phi (N x k, where k is the number of topics) are the multinomial paramters.  See page 1004 of paper for derivation.

In [6]:
def Estep(k, d, alpha, beta, corpusMatrix, tol):    
    
    #storing the total number of words and the number of unique words
    N = corpusMatrix[0][d].shape[0]
    V = corpusMatrix[0][d].shape[1]
    
    #initialize phi and gamma
    oldPhi  = np.full(shape = (N,k), fill_value = 1/k)
    gamma = alpha + N/k
    newPhi = oldPhi
    converge = 0 
    
    
    count = 0
    
    while converge == 0:
        newPhi  = np.zeros(shape = (N,k))
        for n in range(0, N):
            for i in range(0,k):
                newPhi[n,i] = (beta[i, list(corpusMatrix[0][d][n,:]).index(1)])*np.exp(scipy.special.psi(gamma[i]))
        newPhi = newPhi/np.sum(newPhi, axis = 1)[:, np.newaxis] #normalizing the rows of new phi

        for i in range(0,k):
            gamma[i] = alpha[i] + np.sum(newPhi[:, i]) #updating gamma


        criteria = (1/(N*k)*np.sum((newPhi - oldPhi)**2))**0.5
        if criteria < tol:
            converge = 1
        else:
            oldPhi = newPhi
            count = count +1
            converge = 0
    return (newPhi, gamma)

In [234]:
k = 3
np.random.seed(7)
alphaOld = 10*np.random.rand(k)
V = corpusMatrix[0][0].shape[1]
betaOld = np.random.rand(k, V)
betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]

In [235]:
%%time 
for i in range(0,100):
    phi, gamma = Estep(k, 0, alphaOld, betaOld, corpusMatrix, 0.1)

CPU times: user 45.8 ms, sys: 1.67 ms, total: 47.5 ms
Wall time: 46 ms


**Parameter Estimation**

**M Step:** In the E step above, we maximized a lower bound with respect to gamma and phi, and in the M step, for fixed values of these variational parameters, we maximize the lower bound of the log likelihood with repsect to alpha and beta to update these values (combined, these two steps give approximate empirical Bayes estimates for the LDA model).  See pg. 1006 and appendix A.2 for derivation.  

The alphaUpdate() function uses the linear Newton-Rhapson method to update the Dirichlet parameters, alpha, while the Mstep() function maximizes for alpha and beta.

In [7]:
#Update alpha using linear Newton-Rhapson Method#
def alphaUpdate(k, M, alphaOld, gamma, tol):
    h = np.zeros(k)
    g = np.zeros(k)
    alphaNew = np.zeros(k)

    converge = 0
    while converge == 0:
        for i in range(0, k):
            docSum = 0
            for d in range(0, M):
                docSum += scipy.special.psi(gamma[d][i]) - scipy.special.psi(np.sum(gamma[d]))
            g[i] = M*(scipy.special.psi(sum(alphaOld)) - scipy.special.psi(alphaOld[i])) + docSum
            h[i] = M*scipy.special.polygamma(1, alphaOld[i])
        z =  -scipy.special.polygamma(1, np.sum(alphaOld))
        c = np.sum(g/h)/(1/z + np.sum(1/h))
        step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            alphaNew = alphaOld + step
            alphaOld = alphaNew

    return alphaNew

In [8]:
def Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol):
    #Calculate beta#
    V = corpusMatrix[0][0].shape[1]
    beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(0,M):
                Nd = corpusMatrix[0][d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi[d][n,i]*corpusMatrix[0][d][n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]
    
    ##Update ALPHA##
    alphaNew = alphaUpdate(k, M, alphaOld, gamma, tol)
    return(alphaNew, beta)

Check that gsl and scipy results are similar
----

In [29]:
! python setup.py build
! python setup.py install

python: can't open file 'setup.py': [Errno 2] No such file or directory
python: can't open file 'setup.py': [Errno 2] No such file or directory


In [28]:
%%cython -lgsl

#from cython_gsl cimport *
import cython_gsl
import scipy

print(gsl_sf_psi_n(1, 10))
print(scipy.special.polygamma(1, 10))
print(gsl_sf_psi(10))
print(scipy.special.psi(10))


Error compiling Cython file:
------------------------------------------------------------
...

#from cython_gsl cimport *
import cython_gsl
import scipy

print(gsl_sf_psi_n(1, 10))
                 ^
------------------------------------------------------------

/Users/annayanchenko/.ipython/cython/_cython_magic_0fedd72c363e1a867df25274b4ecc244.pyx:6:18: undeclared name not builtin: gsl_sf_psi_n

Error compiling Cython file:
------------------------------------------------------------
...
import cython_gsl
import scipy

print(gsl_sf_psi_n(1, 10))
print(scipy.special.polygamma(1, 10))
print(gsl_sf_psi(10))
               ^
------------------------------------------------------------

/Users/annayanchenko/.ipython/cython/_cython_magic_0fedd72c363e1a867df25274b4ecc244.pyx:8:16: undeclared name not builtin: gsl_sf_psi


Cython code for updateing $\alpha$ and $\beta$
----

In [14]:
%%cython -a -lgsl
from cython_gsl cimport *


Error compiling Cython file:
------------------------------------------------------------
...
from cython_gsl cimport *
^
------------------------------------------------------------

/Users/annayanchenko/.ipython/cython/_cython_magic_188f1a7ce08075e9fd1c3394fc79638d.pyx:1:0: 'cython_gsl.pxd' not found


In [9]:
%%cython -a -lgsl

import cython
import numpy as np
cimport numpy as np
import scipy
from cython_gsl cimport *

#Update alpha using linear Newton-Rhapson Method#

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:] cy_alphaUpdate(int k, int M, double[:] alphaOld, double[:, :] gamma, double tol):
    cdef double[:] h = np.zeros(k)
    cdef double[:] g = np.zeros(k)
    cdef double[:] alphaNew = np.zeros(k)

    cdef int converge
    cdef int i, d
    cdef double docSum
    cdef double[:] step = np.zeros(k)
    cdef double c, s1, s2

    cdef double alpha_sum = 0
    for i in range(k):
        alpha_sum += alphaOld[i]

    converge = 0
    while converge == 0:
        z = -gsl_sf_psi_n(1, alpha_sum)

        s1 = 0
        s2 = 1.0/z
        for i in range(0, k):
            docSum = 0
            for d in range(M):
                docSum += gsl_sf_psi(gamma[d,i]) - gsl_sf_psi(sum(gamma[d]))
            g[i] = M*(gsl_sf_psi(alpha_sum) - gsl_sf_psi(alphaOld[i])) + docSum
            h[i] = M*gsl_sf_psi_n(1, alphaOld[i])
            
            s1 += g[i]/h[i]
            s2 += 1.0/h[i]
        c = s1/s2

        for i in range(k):
            step[i] = (g[i] - c)/h[i]
        # step = (g - c)/h
        if np.linalg.norm(step) < tol:
            converge = 1
        else:
            converge = 0
            for i in range(k):
                alphaNew[i] = alphaOld[i] + step[i]
            alphaOld = alphaNew

    return alphaNew

@cython.wraparound(False)
@cython.boundscheck(False)
cpdef double[:, :] cy_betaUpdate(int k, int M, long[:] phi_1, double[:, :, :] phi_2, double[:, :] gamma, 
                              long[:] c_1, double[:, :, :] c_2, double tol):

    cdef int i, j, d, n
    cdef int Nd
    cdef double wordSum

    #Calculate beta#
    cdef int V = c_2[0].shape[1]
    cdef double[:, :] beta = np.zeros(shape = (k,V))

    for i in range(0,k):
        for j in range(0,V):
            wordSum = 0
            for d in range(M):
                Nd = c_1[d] # c[d].shape[0]
                for n in range(0, Nd):
                    wordSum += phi_2[d, n,i]*c_2[d, n,j]
            beta[i,j] = wordSum
    #Normalize the rows of beta#
    beta = beta/np.sum(beta, axis = 1)[:, np.newaxis]

    return beta

Check Cython functions against original functions
----

### Prepare funciton arguments

In [10]:
k = 3
M = 4
phi = [np.array([[ 0.38340797,  0.30327435,  0.31331768],
       [ 0.5703127 ,  0.17742331,  0.25226399],
       [ 0.3176045 ,  0.43558053,  0.24681497]]), np.array([[ 0.55072546,  0.18583053,  0.26344401],
       [ 0.30048528,  0.44698122,  0.2525335 ],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608],
       [ 0.32099179,  0.28895213,  0.39005608]]), np.array([[ 0.0623253 ,  0.24856995,  0.68910475],
       [ 0.63063084,  0.08148213,  0.28788703],
       [ 0.12858014,  0.39114008,  0.48027977]]), np.array([[ 0.66366397,  0.08257639,  0.25375964],
       [ 0.77627072,  0.10684157,  0.11688771],
       [ 0.75940435,  0.23665379,  0.00394186],
       [ 0.02553949,  0.62915851,  0.345302  ],
       [ 0.14168349,  0.41504784,  0.44326867]])]
gamma = np.array([np.array([ 9.71859516,  6.63758257,  7.30234283]), np.array([ 10.58244789,   7.50992466,   8.56614802]), np.array([ 9.26880627,  6.44249655,  7.94721775]), np.array([ 10.813832  ,   7.19158249,   7.65310607])])
alphaOld = np.array([ 8.44726999 , 5.72130438,  6.48994619])
c = np.array([np.array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]), np.array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])])
tol = 0.1

In [11]:
phi

[array([[ 0.38340797,  0.30327435,  0.31331768],
        [ 0.5703127 ,  0.17742331,  0.25226399],
        [ 0.3176045 ,  0.43558053,  0.24681497]]),
 array([[ 0.55072546,  0.18583053,  0.26344401],
        [ 0.30048528,  0.44698122,  0.2525335 ],
        [ 0.32099179,  0.28895213,  0.39005608],
        [ 0.32099179,  0.28895213,  0.39005608],
        [ 0.32099179,  0.28895213,  0.39005608],
        [ 0.32099179,  0.28895213,  0.39005608]]),
 array([[ 0.0623253 ,  0.24856995,  0.68910475],
        [ 0.63063084,  0.08148213,  0.28788703],
        [ 0.12858014,  0.39114008,  0.48027977]]),
 array([[ 0.66366397,  0.08257639,  0.25375964],
        [ 0.77627072,  0.10684157,  0.11688771],
        [ 0.75940435,  0.23665379,  0.00394186],
        [ 0.02553949,  0.62915851,  0.345302  ],
        [ 0.14168349,  0.41504784,  0.44326867]])]

In [12]:
r = len(phi)
phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
n = max(phi_1)
p = phi[0].shape[1]
phi_2 = np.zeros((r, n, p))
for i, j in enumerate(phi_1):
    phi_2[i, :j, :] = phi[i]

In [13]:
c

array([ array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]]),
       array([[ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.]]),
       array([[ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]]),
       array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  1.

In [14]:
r = len(c)
c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
n = max(c_1)
p = c[0].shape[1]
c_2 = np.zeros((r, n, p))
for i, j in enumerate(c_1):
    c_2[i, :j, :] = c[i]

### Check

In [15]:
a, b = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)

In [16]:
a

array([ 8.66099489,  5.98037246,  6.76611379])

In [17]:
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
alphaNew

array([ 8.43757551,  5.82900254,  6.59352755])

In [18]:
b

array([[ 0.16999332,  0.19626581,  0.04098256,  0.09372663,  0.19469974,
         0.00945096,  0.11515546,  0.11771306,  0.05813967,  0.00387279],
       [ 0.07418842,  0.03350616,  0.16465017,  0.18024822,  0.23605422,
         0.05076618,  0.04833251,  0.02182057,  0.06193863,  0.12849492],
       [ 0.0936114 ,  0.09831979,  0.16764266,  0.09064182,  0.28321217,
         0.12508641,  0.00071553,  0.02121748,  0.05687348,  0.06267927]])

In [19]:
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
betaNew

array([[ 0.16999332,  0.19626581,  0.04098256,  0.09372663,  0.19469974,
         0.00945096,  0.11515546,  0.11771306,  0.05813967,  0.00387279],
       [ 0.07418842,  0.03350616,  0.16465017,  0.18024822,  0.23605422,
         0.05076618,  0.04833251,  0.02182057,  0.06193863,  0.12849492],
       [ 0.0936114 ,  0.09831979,  0.16764266,  0.09064182,  0.28321217,
         0.12508641,  0.00071553,  0.02121748,  0.05687348,  0.06267927]])

Is there any speed-up?
----

In [20]:
%timeit a, b = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)

1000 loops, best of 3: 1.34 ms per loop


In [21]:
%%timeit
alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))

1000 loops, best of 3: 137 µs per loop


**LDA Function:**
Finally, we implement the entire Latent Dirichlet Allocation method in the LDA function, which takes as its arguments k (the number of topics), D (the number of documents in the corpus), a corpus matrix (the output from make_word_matrix above) and a tolerance (which sets the convergence criteria for the while loops).  For each document d, the function runs until the alpha or beta parameters converge, by first running the E step and then the M step for each document separately.  The final values of phi, gamma, alpha and beta are returned for all D documents in a list.

In [28]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDA(k, corpusMatrix, tol):

    # create rectangular matrices for c
    c = corpusMatrix[0]
    r_c = len(c)
    c_1 = np.array([c_.shape[0] for c_ in c]).astype('int')
    n_c = max(c_1)
    p_c = c[0].shape[1]
    c_2 = np.zeros((r_c, n_c, p_c))
    for i, j in enumerate(c_1):
        c_2[i, :j, :] = c[i]
    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    
    while converge == 0:
        phi = []
        gamma = []
        #looping through the number of documents
        for d in range(0,M): #M is the number of documents
            phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
            phi.append(phiT)
            gamma.append(gammaT)
            
        # create rectangular matrices for phi
        r = len(phi)
        phi_1 = np.array([p_.shape[0] for p_ in phi]).astype('int')
        n = max(phi_1)
        p = phi[0].shape[1]
        phi_2 = np.zeros((r, n, p))
        for i, j in enumerate(phi_1):
            phi_2[i, :j, :] = phi[i]

                
        gamma = np.array(gamma)

        alphaNew = np.array(cy_alphaUpdate(k, M, alphaOld, gamma, tol))
        betaNew = np.array(cy_betaUpdate(k, M, phi_1, phi_2, gamma, c_1, c_2, tol))
    
        if np.linalg.norm(alphaOld - alphaNew) < tol or np.linalg.norm(betaOld - betaNew) < tol:
            converge =1
        else: 
            converge =0
            alphaOld = alphaNew
            betaOld = betaNew
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output

In [25]:
%%time

LDA(3, corpusMatrix, 0.1, 1)

CPU times: user 3.64 ms, sys: 518 µs, total: 4.16 ms
Wall time: 3.8 ms


[[[array([[ 0.14642529,  0.68443033,  0.16914438],
          [ 0.47795273,  0.24350816,  0.27853911],
          [ 0.31986014,  0.65907976,  0.0210601 ]]),
   array([[ 0.51746066,  0.19758617,  0.28495316],
          [ 0.3836557 ,  0.59247515,  0.02386915],
          [ 0.65344173,  0.07082137,  0.2757369 ],
          [ 0.65344173,  0.07082137,  0.2757369 ],
          [ 0.65344173,  0.07082137,  0.2757369 ],
          [ 0.65344173,  0.07082137,  0.2757369 ]]),
   array([[ 0.28080277,  0.48860678,  0.23059045],
          [ 0.10181848,  0.78482247,  0.11335905],
          [ 0.04628882,  0.77639317,  0.177318  ]]),
   array([[ 0.10423655,  0.77301779,  0.12274566],
          [ 0.24569688,  0.21721047,  0.53709266],
          [ 0.37436505,  0.36139955,  0.2642354 ],
          [ 0.30274009,  0.53267172,  0.16458819],
          [ 0.04719444,  0.76158964,  0.19121592]])],
  array([[  8.32423203,  11.34875111,   5.32745725],
         [ 10.89487714,  10.83507967,   6.27048357],
         [  7.8089

In [34]:
#function to return the most probable words
#p is the number of words you want returned for each topic
def mostCommon(beta, wordList, p):
    k = beta.shape[0]
    topicWords = []
    betaDF = pd.DataFrame(beta)
    betaDF.columns = wordList
    for i in range(0, k):
        document = betaDF.loc[i,:]
        document.sort(1, ascending = 0)
        mostCommon = pd.DataFrame(document[0:p])
        topicWords.append(mostCommon)
    return(topicWords)

In [16]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health." 
docsTest = collections.OrderedDict()
docsTest["s1"] = doc_a
docsTest["s2"] = doc_b
docsTest["s3"] = doc_c
docsTest["s4"] = doc_d 
docsTest["s5"] = doc_e 
for k, v in docsTest.items():
    print(k,v)
    

s1 Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother.
s2 My mother spends a lot of time driving my brother around to baseball practice.
s3 Some health experts suggest that driving may cause increased tension and blood pressure.
s4 I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better.
s5 Health professionals say that brocolli is good for your health.


In [17]:
cTest = make_word_matrix(docsTest, 1)

**State of the Union Example**

In order to determine the initial performance time of our algorithm, we implemented LDA on a small corpus of eight documents. These documents are each of the State of the Union Adresses delivered by President Clinton during his time in office. These were found in the nltk Python package.

In [26]:
import nltk
#nltk.download("state_union")
from nltk.corpus import state_union

In [27]:
#creating a diciontary of the state of the Union Addresses for Clinton
fileNames = state_union.fileids()
Clinton = fileNames[50:58]

ClintonSOTU = {}
for i in range(0, len(Clinton)):
    ClintonSOTU[Clinton[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(Clinton[i]))

In [30]:
fileNames = state_union.fileids()
SOTU = fileNames

PresSOTU = {}
for i in range(0, len(SOTU)):
    PresSOTU[SOTU[i].rsplit('.', 1)[0]] = list(nltk.corpus.state_union.words(SOTU[i]))

In [32]:
np.random.seed(17)
PresCorp = make_word_matrix(PresSOTU, 0)
ResultsAll = LDA(5, PresCorp, 0.1)

In [29]:
%%time
np.random.seed(7)
ClintCorp = make_word_matrix(ClintonSOTU, 0)
Results = LDA(3, ClintCorp, 0.1)

CPU times: user 1min 46s, sys: 1.84 s, total: 1min 48s
Wall time: 1min 49s


In [39]:
mostCommon(ResultsAll[0][3], PresCorp[1], 2)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[                 0
 american  0.016356
 nation    0.012914,                1
 nation  0.012595
 can     0.010289,               2
 will   0.031106
 peopl  0.012619,              3
 year  0.021055
 will  0.016146,              4
 year  0.016371
 will  0.016083]

In [40]:
from nltk.corpus import gutenberg
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [54]:
bookNames = gutenberg.fileids()
indicies = [0,3,7,15]
books = [bookNames[i] for i in indicies]
books

['austen-emma.txt',
 'bible-kjv.txt',
 'carroll-alice.txt',
 'shakespeare-hamlet.txt']

In [55]:


booksInput = {}
for i in range(0, len(books)):
    booksInput[books[i].rsplit('.', 1)[0]] = list(nltk.corpus.gutenberg.words(books[i]))

In [ ]:
np.random.seed(77)
bookCorp = make_word_matrix(booksInput, 0)
ResultsBooks = LDA(5, bookCorp, 0.1)

In [49]:
ResultsAll[0][3]

array([[  2.09616476e-03,   7.21814172e-05,   1.79853715e-05, ...,
          3.81446711e-04,   1.67104484e-05,   3.01931496e-06],
       [  7.53196999e-04,   1.48237362e-05,   2.55025962e-05, ...,
          1.13674430e-04,   2.93999946e-07,   3.68877172e-05],
       [  6.79357855e-04,   7.35381859e-05,   3.37436041e-05, ...,
          4.83181647e-04,   1.96270859e-05,   3.56282286e-05],
       [  1.36895570e-03,   4.50351974e-06,   2.53831275e-05, ...,
          1.08060550e-03,   4.11311137e-05,   1.13503171e-05],
       [  2.70702937e-03,   9.20969443e-05,   2.58135710e-05, ...,
          1.25764501e-04,   5.12991407e-05,   4.16335666e-05]])

In [50]:
mostCommon(ResultsAll[0][3], bookCorp[1], 5)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[               0
 d       0.022322
 ham     0.016085
 caesar  0.010088
 shall   0.010001
 hath    0.008128,              1
 thou  0.019477
 s     0.016152
 lord  0.011784
 will  0.010453
 ham   0.010397,               2
 d      0.021710
 will   0.020551
 thou   0.015766
 shall  0.012959
 haue   0.011819,                3
 d       0.023517
 haue    0.019264
 lord    0.012216
 caesar  0.009397
 come    0.009008,              4
 haue  0.015033
 vs    0.010743
 come  0.010299
 d     0.009763
 will  0.009447]

In [ ]:
#pip install --pre line-profiler
#pip install psutil
# pip install memory_profiler

In [ ]:
#creating profile files for the construction of the word matrix and LDA for SOTU example
%prun -q -D ClintonWordMatrix.prof make_word_matrix(ClintonSOTU,0)
%prun -q -D ClintonLDA.prof LDA(3, ClintCorp, 0.1, 0)

In [ ]:
#printing profile results
import pstats
wordMatrixProfile = pstats.Stats('ClintonWordMatrix.prof')
LDAProfile = pstats.Stats('ClintonLDA.prof')
wordMatrixProfile.print_stats()
LDAProfile.print_stats()
pass

**Movie Prediction** 

In addition to text analysis, LDA can be implemented for other problems that have the same structure as a text corpora. In this section, we use LDA in order to predict movies preferred by users of the site MovieLens.com. The goal is to predict another movie that the user likes based on the movies that they have already said that they prefer. The definition of preferred here is that the user rated the movie four or five, where five is the maxiumum possible rating. Users who gave a preferred rating to at least 50 movies were used. A training set of 2,535 users was used to fit the model. The model is then shown all but one of the preferred movies for the users in the test data, and this is used to determine the last movie that the user preferred. Thus, it is possible to evaluate the performance of the model.

In [ ]:
#reading in the data
import pandas as pd
trainRatings = pd.read_csv("UserRatingsTraining.csv")
testRatings = pd.read_csv("UserRatingsTest.csv")

In [ ]:
#making a dictionary for the training and testing data
trainingUsers = list(set(trainRatings['userID']))
trainingDict = {}
for i in range(0, len(trainingUsers)):
    movies = list(map(str, list(pd.DataFrame(trainRatings[trainRatings['userID'] == trainingUsers[0]])['movieID'])))
    trainingDict[trainingUsers[i]] = movies
    
testUsers = list(set(testRatings['userID']))
testDict = {}
for i in range(0, len(testUsers)):
    movies = list(map(str, list(pd.DataFrame(testRatings[testRatings['userID'] == testUsers[0]])['movieID'])))
    testDict[testUsers[i]] = movies

In [ ]:
#running LDA analysis to get parameters for movie training data
movieMatrix = make_word_matrix(trainingDict, 0)
movieOutput = LDA(3, 4, movieMatrix, 0.1, 0)

In [ ]:
#use the above parameters to make predictions for the test set
#hold out one movie randomly, then determine the likelihood of the possible held out movies
#the highest likelihood should be assigned to the last movie, if the model works correctly

## Future Steps

- Compare output to Python package
- Test on corpus in paper
- Model topics on different corpus
- Time and optimize (use Cython, quite slow now)
- Run collaborative filtering
- Compare to Gibbs sampling

## Compare our output to Python Package

https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

In [43]:
#! pip install gensim

In [ ]:
p_stemmer = PorterStemmer()
for i in docs:
    stemmed_tokens = p_stemmer.stem(i)

In [66]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

tokenizer = RegexpTokenizer(r'\w+')

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()
    
# create sample documents
doc_a = "The quick brown fox"
doc_b = "Brown fox jumps over the jumps jumps jumps"
doc_c =  "The the the lazy dog elephant."
doc_d = "The the the the the dog peacock lion tiger elephant"


# compile sample documents into a list
doc_set = [doc_a, doc_b, doc_c, doc_d]

# list for tokenized documents in loop
texts = []

# loop through document list
for i in doc_set:
    
    # clean and tokenize document string
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [i for i in tokens if not i in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=3, id2word = dictionary, iterations = 1)

In [67]:
print(ldamodel.print_topics(num_topics=3, num_words=4))

[(0, '0.208*jump + 0.122*fox + 0.106*brown + 0.105*dog'), (1, '0.199*jump + 0.128*brown + 0.104*fox + 0.103*eleph'), (2, '0.148*jump + 0.133*dog + 0.131*eleph + 0.108*fox')]


In [69]:
beta =  LDATest(3, corpusMatrix, 0.1, 1)[0][3]

In [70]:
mostCommon(beta, corpusMatrix[1], 4)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:10: FutureWarning: sort is deprecated, use sort_values(inplace=True) for for INPLACE sorting


[              0
 jump   0.502213
 dog    0.233400
 brown  0.073339
 lion   0.050304,               1
 fox    0.180265
 jump   0.163142
 brown  0.137126
 dog    0.135596,                 2
 jump     0.239419
 eleph    0.143304
 peacock  0.140478
 lion     0.133684]

In [52]:
corpusMatrix[1]

['brown',
 'dog',
 'eleph',
 'fox',
 'jump',
 'lazi',
 'lion',
 'peacock',
 'quick',
 'tiger']

In [53]:
beta

array([[  4.62560896e-34,   3.33343130e-01,   3.33343094e-01,
          3.30440861e-34,   3.27807481e-36,   3.33313776e-01,
          1.55467573e-14,   1.53680851e-14,   1.26552889e-33,
          5.71817087e-15],
       [  1.09742914e-54,   1.99989427e-01,   1.99989449e-01,
          1.97699403e-54,   1.67679702e-57,   1.84020254e-56,
          2.00007041e-01,   2.00007041e-01,   2.83838287e-51,
          2.00007041e-01],
       [  2.22222222e-01,   4.02204997e-33,   1.14123694e-32,
          2.22222222e-01,   4.44444444e-01,   1.02851237e-35,
          2.11185563e-38,   1.71351548e-38,   1.11111111e-01,
          9.35199488e-40]])

In [68]:
#k = number of topics, D = number of documents#
#corpus matrix is output of make_word_matrix# 
def LDATest(k, corpusMatrix, tol, needToSplit):

    
    ##Check for proper input##
    if isinstance(k, int) != True or k <= 0:
        print("Number of topics must be a positive integer")
        return;
    
    if tol <=0:
        print("Convergence tolerance must be positive")
        return;
    
    if needToSplit != 0 and needToSplit != 1:
        print("NeedToSplit argument must be 0 or 1")
        return;
    
    
    
    M = corpusMatrix[2]
    output = []
    
    converge = 0
    iterations = 0
    #initialize alpha and beta for first iteration
    alphaOld = 10*np.random.rand(k)
    V = corpusMatrix[0][0].shape[1]
    betaOld = np.random.rand(k, V)
    betaOld = betaOld/np.sum(betaOld, axis = 1)[:, np.newaxis]
    

    phi = []
    gamma = []
    #looping through the number of documents
    for d in range(0,M): #M is the number of documents
        phiT, gammaT = Estep(k, d, alphaOld, betaOld, corpusMatrix, tol)
        phi.append(phiT)
        gamma.append(gammaT)

    alphaNew, betaNew = Mstep(k, M, phi, gamma, alphaOld, corpusMatrix, tol)


    converge =0
    alphaOld = alphaNew
    betaOld = betaNew
    iterations += iterations
    output.append([phi, gamma, alphaNew, betaNew])
        
    return output